In [8]:
import gymnasium as gym
import numpy as np
import random

In [9]:
learning_rate = 0.1
discount = 0.95

epsilon = 0.5
decay = 0.98

num_episodes = 25000

render = False

if render:
    env = gym.make("MountainCar-v0", render_mode='human')

else:
    env = gym.make("MountainCar-v0")
env.reset()




(array([-0.5023207,  0.       ], dtype=float32), {})

In [10]:
num_actions = env.action_space.n

discrete_obs_space_size = [20] * len(env.observation_space.high)

discrete_obs_space_window_size = (env.observation_space.high - env.observation_space.low) / discrete_obs_space_size

q_table = np.random.uniform(low = -2, high = 0, size = (discrete_obs_space_size + [num_actions]))



In [11]:
def get_disc_state(state):
    discrete_state = (state - env.observation_space.low) / discrete_obs_space_window_size
    return tuple(discrete_state.astype(int))


In [12]:
discrete_state = get_disc_state(env.reset()[0])

In [13]:
for episode in range(num_episodes):

    done = False
    while not done:


        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, num_actions)

        epsilon *= decay

        action = np.argmax(q_table[discrete_state])

        new_state, reward, termination, truncation, _ = env.step(action)

        done = termination

        new_discrete_state = get_disc_state(new_state)

        

        if not done:
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action, )]

            new_q = (1 - learning_rate) * current_q * learning_rate * (reward + discount * max_future_q)

            q_table[discrete_state + (action, )] = new_q

        elif new_state[0] >= env.goal_position:
            print (f"We made it on episode {episode}")
            q_table[discrete_state + (action,)] = 0

        discrete_state = new_discrete_state




